Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = @__DIR__
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);

first(df, 5)

,actor,recipient,condition,block,trial,prosoc_left,chose_prosoc,pulled_left
,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,1,NA,0,1,2,0,1,0
2,1,NA,0,1,4,0,0,1
3,1,NA,0,1,6,1,0,0
4,1,NA,0,1,8,0,1,0
5,1,NA,0,1,10,1,1,1


Define the Stan language model

In [4]:
m_10_04 = "
data{
    int N;
    int N_actors;
    int pulled_left[N];
    int prosoc_left[N];
    int condition[N];
    int actor[N];
}
parameters{
    vector[N_actors] a;
    real bp;
    real bpC;
}
model{
    vector[N] p;
    bpC ~ normal( 0 , 10 );
    bp ~ normal( 0 , 10 );
    a ~ normal( 0 , 10 );
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + (bp + bpC * condition[i]) * prosoc_left[i];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [5]:
stanmodel = Stanmodel(name="m_10_04",
model=m_10_04, output_format=:mcmcchains);


File /Users/rob/.julia/dev/StanModels/notebooks/10/tmp/m_10_04.stan will be updated.



Input data for cmdstan

In [6]:
m_10_04_data = Dict("N" => size(df, 1), "N_actors" => length(unique(df[:actor])),
"actor" => df[:actor], "pulled_left" => df[:pulled_left],
"prosoc_left" => df[:prosoc_left], "condition" => df[:condition]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_10_04_data, ProjDir, diagnostics=false,
  summary=true, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
      mean   sd  5.5% 94.5% n_eff Rhat
bp    0.84 0.26  0.43  1.26  2271    1
bpC  -0.13 0.29 -0.59  0.34  2949    1

a[1] -0.74 0.27 -1.16 -0.31  3310    1
a[2] 10.88 5.20  4.57 20.73  1634    1
a[3] -1.05 0.28 -1.52 -0.59  4206    1
a[4] -1.05 0.28 -1.50 -0.60  4133    1
a[5] -0.75 0.27 -1.18 -0.32  4049    1
a[6]  0.22 0.27 -0.22  0.65  3877    1
a[7]  1.81 0.39  1.22  2.48  3807    1
";


Inference for Stan model: m_10_04_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (1.6, 1.5, 1.4, 1.5) seconds, 6.0 seconds total
Sampling took (1.4, 1.4, 1.4, 1.4) seconds, 5.5 seconds total

                 Mean     MCSE  StdDev     5%    50%    95%  N_Eff  N_Eff/s    R_hat
lp__             -261  5.1e-02     2.1   -265   -261   -259   1769      324  1.0e+00
accept_stat__    0.86  3.4e-03    0.20   0.41   0.93    1.0   3429      629  1.0e+00
stepsize__       0.35  7.2e-03   0.010   0.34   0.36   0.37    2.0     0.37  2.0e+13
treedepth__       3.3  1.0e-02    0.62    2.0    3.0    4.0   3694      677  1.0e+00
n_leapfrog__       12  1.2e-01     7.1    7.0     15     23   3668      673  1.0e+00
divergent__     0.065  4.3e-03    0.25   0.00   0.00    1.0   3295      604  1.0e+00
energy__          266  7.6e-02     3.0    262    266    271   1573      289  1.0e+00
a[1]            -0.74  5.3e-03    0.27   -1.2  -0.

Update sections

In [8]:
chn2 = set_section(chn, Dict(
  :parameters => ["bp", "bpC"],
  :pooled => ["a.$i" for i in 1:7],
  :internals => ["lp__", "accept_stat__", "stepsize__", "treedepth__", "n_leapfrog__",
    "divergent__", "energy__"]
  )
)

Object of type Chains, with data of type 1000×16×4 Array{Float64,3}

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
pooled            = a.1, a.2, a.3, a.4, a.5, a.6, a.7
internals         = lp__, accept_stat__, stepsize__, treedepth__, n_leapfrog__, divergent__, energy__
parameters        = bp, bpC

parameters
      Mean    SD   Naive SE  MCSE   ESS
 bp  0.8342 0.2603   0.0041 0.0055 1000
bpC -0.1360 0.2966   0.0047 0.0060 1000



Describe parameter draws

In [9]:
describe(chn2)

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = bp, bpC

Empirical Posterior Estimates
─────────────────────────────────────────
parameters
      Mean    SD   Naive SE  MCSE   ESS
 bp  0.8342 0.2603   0.0041 0.0055 1000
bpC -0.1360 0.2966   0.0047 0.0060 1000

Quantiles
─────────────────────────────────────────
parameters
      2.5%   25.0%   50.0%   75.0%  97.5%
 bp  0.3146  0.6577  0.8345 1.0090 1.3558
bpC -0.7229 -0.3329 -0.1335 0.0654 0.4486



Describe pooled parameter draws

In [10]:
describe(chn2, section=:pooled)

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
pooled            = a.1, a.2, a.3, a.4, a.5, a.6, a.7

Empirical Posterior Estimates
───────────────────────────────────────────
pooled
      Mean    SD   Naive SE  MCSE   ESS
a.1 -0.7415 0.2713   0.0043 0.0053 1000
a.2 11.0413 5.3127   0.0840 0.1178 1000
a.3 -1.0479 0.2847   0.0045 0.0049 1000
a.4 -1.0468 0.2819   0.0045 0.0063 1000
a.5 -0.7289 0.2633   0.0042 0.0048 1000
a.6  0.2273 0.2684   0.0042 0.0047 1000
a.7  1.8182 0.3989   0.0063 0.0072 1000

Quantiles
───────────────────────────────────────────
pooled
      2.5%   25.0%   50.0%   75.0%   97.5% 
a.1 -1.2856 -0.9192 -0.7390 -0.5535 -0.2263
a.2  3.9468  6.9202  9.8757 14.1964 23.7509
a.3 -1.6256 -1.2349 -1.0449 -0.8576 -0.5016
a.4 -1.5980 -1.2401 -1.0411 -0.8609 -0.4810
a.5 -1.2691 -0.9016 -0.7247 -0.5494 -0.2307
a.6 -0.3122  0.0498  0.2265  0.4046  0.7640
a.7  1.0827  1.5398  1.8023  2.0751  2.6671



Make it a DataFrame

In [11]:
df = DataFrame(chn2, [:parameters, :pooled])

,bp,bpC,a.1,a.2,a.3,a.4,a.5,a.6,a.7
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.685677,0.097135,-0.745356,11.6086,-1.09798,-0.874046,-0.739401,0.0809897,1.57253
2,1.02894,-0.459716,-0.929179,4.30027,-0.913189,-1.0982,-0.713167,0.601966,2.45767
3,0.457449,0.00354925,-0.775134,13.3544,-0.647044,-0.818562,-0.522957,0.298212,1.71648
4,0.617577,0.218592,-0.508346,5.43116,-1.51773,-1.03842,-0.850773,0.248294,1.59961
5,0.947153,-0.402022,-0.699986,14.9541,-0.696363,-0.981015,-0.719213,0.0101301,3.00923
6,0.675831,-0.00187811,-0.299711,19.5546,-1.05235,-1.51161,-1.06108,0.0985422,1.94251
7,0.607533,-0.329485,-0.546325,14.1924,-0.962261,-0.422542,-0.0782751,0.633032,1.93488
8,0.865398,-0.836392,-0.872458,10.1876,-0.741429,-0.751142,-0.281395,0.801744,1.94114
9,0.923527,0.0436333,-1.33449,5.66693,-1.35219,-1.50298,-0.580775,0.409871,1.69153


End of `10/m10.04s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*